In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import spacy

In [28]:
## Read in the three books ##
def read_book(num):
    names = ['1_Inkheart.txt', '2_Inkspell.txt', '3_Inkdeath.txt']
    with open('./books/'+names[num], 'r') as f:
        book = f.read().splitlines()
        f.close()
    book = [line for line in book if line != '']
    return book


## For each book, seperate it in chapters ##
def chop_chapters(book, reg=r'[0-9]+'):
    chapters = {}

    first_line = 0
    chap_count = 0
    for count, line in enumerate(book):
        tmp = re.match(reg, line)
        if tmp is not None:
            chapters[chap_count] = book[first_line:count]
            first_line = count
            chap_count += 1
        elif count == len(book)-1:
            chapters[chap_count] = book[first_line:count+1]
            first_line = count
            chap_count += 1
    return chapters


# The name of the book is a recurring line which we don't want to take into account
def remove_book_name(book, name):
    return [line for line in book if name not in line]


# In the third book, each chapter begins with the name of the chapter
# something that we also don't want to include in our text processing
def remove_chapter_name(chap_dict):
    for chap_num, chapter in chap_dict.items():
        # print(chapter[:2], '|', chapter[2:])
        chap_dict[chap_num] = chapter[2:]


In [29]:
# We read every book and its chapters
book1 = read_book(0)
book2 = read_book(1)
book3 = read_book(2)

# We remove the book name references
book2 = remove_book_name(book2, 'Ink 2 - Inkspell')
book3 = remove_book_name(book3, 'Ink 3 - Inkdeath')

# We divide each book in chapters (where book 3 has different chapter names)
ch1 = chop_chapters(book1)
ch2 = chop_chapters(book2)
ch3 = chop_chapters(book3, reg=r'CHAPTER [0-9]+')

# And for the third book we remove the chapter names
ch3 = remove_chapter_name(ch3)

In [30]:
# In the first book, each chapter starts with a poem or quote
# The author is always quoted with '- [author]' so we can separate it easily
for ch_count, chapter in ch1.items():
    for count, line in enumerate(chapter):
        if line[0] == '-':
            # print(ch_count, '\n', chapter[count] + '\n', chapter[count+1:])
            # We set the chapter to the text WITHOUT the poem
            ch1[ch_count] = chapter[count+1:]
            # No need to check more lines in this chapter
            break

In [31]:
for ch_count, chapter in ch2.items():
    for count, line in enumerate(chapter):
        if line[0] == '-':
            # print(ch_count, '\n', chapter[count] + '\n', chapter[count+1:])
            ch2[ch_count] = chapter[count+1:]
            break



In [32]:
for ch_count, chapter in ch3.items():
    if ch_count == 1:
        for count, line in enumerate(chapter):
            print(line)


CHAPTER 1
NOTHING BUT A DOG AND A SHEET OF PAPER
Moonlight fell on Elinor’s bathrobe, her nightdress, her bare feet, and the dog lying in front of them. Orpheus’s dog. Oh, the way he looked at her with his eternally sad eyes! As if asking himself why, in the name of all the exciting smells in the world, she was sitting in her library in the middle of the night, surrounded by silent books, just staring into space.
“Why?” said Elinor in the silence. “Because I can’t sleep, you stupid animal.” But she patted his head all the same. This is what you’ve come to, Elinor, she thought as she hauled herself out of her armchair. Spending your nights talking to a dog. You don’t even like dogs, least of all this one, with his heavy breathing that always reminds you of his appalling master!
Still, she had kept the dog in spite of the painful memories he brought back. She’d kept the chair, too, even though the Magpie had sat in it. Mortola . . . how often Elinor thought she heard the old woman’s voic

In [50]:
import random
## Check everything we did above ##
for ch in [ch1, ch2, ch3]:
    random_chap = random.randint(0, len(ch))
    print('Chapter {}'.format(random_chap), ch[random_chap][:2])

Chapter 56 ['In books hatred is often described as hot, but at Capricorn\'s festivities Meggie discovered it was cold - an ice-cold hand that stops the heart and presses it like a clenched fist against the ribs. Hatred made her freeze, in spite of the mild air wafting around her, telling her that the world was a good, safe place. She knew it was not - as the bloody cloth on which the smiling Capricorn had laid his ringed hand showed all too clearly. "Well, so much for that!" he cried. "And now for the real reason we are all gathered here tonight. Not only are we about to punish the traitors but we\'re also going to celebrate a reunion with an old friend. Some of you may remember him, and as for the others, I promise that once you have met him you will never forget him."', "Cockerell twisted his thin face into a sour smile. He was obviously not looking forward to the reunion and, at Capricorn's words, alarm showed on several other faces."]
Chapter 69 ['It was the last night before the d